In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [2]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [3]:
estados = ['https://www.fapesc.sc.gov.br/category/chamadas-abertas/', 'https://www.fapesc.sc.gov.br/category/chamadas-encerradas/', 'https://www.fapesc.sc.gov.br/category/chamadas-em-andamento/']

In [12]:
links_pages_estados = []

for estado in estados:

    parser_estado = parser(estado)

    ult_page = parser_estado.xpath('//div[@class="pagination"]//@href')[-1].split('page/')[1].split('/')[0]

#####################

    base = estado

    for i in range(0, int(ult_page)):
        link_estado = base +'page/' + str(i+1) + '/'
        links_pages_estados.append(link_estado)


links_pages_estados

['https://www.fapesc.sc.gov.br/category/chamadas-abertas/page/1/',
 'https://www.fapesc.sc.gov.br/category/chamadas-abertas/page/2/',
 'https://www.fapesc.sc.gov.br/category/chamadas-encerradas/page/1/',
 'https://www.fapesc.sc.gov.br/category/chamadas-encerradas/page/2/',
 'https://www.fapesc.sc.gov.br/category/chamadas-encerradas/page/3/',
 'https://www.fapesc.sc.gov.br/category/chamadas-encerradas/page/4/',
 'https://www.fapesc.sc.gov.br/category/chamadas-encerradas/page/5/',
 'https://www.fapesc.sc.gov.br/category/chamadas-encerradas/page/6/',
 'https://www.fapesc.sc.gov.br/category/chamadas-encerradas/page/7/',
 'https://www.fapesc.sc.gov.br/category/chamadas-encerradas/page/8/',
 'https://www.fapesc.sc.gov.br/category/chamadas-encerradas/page/9/',
 'https://www.fapesc.sc.gov.br/category/chamadas-encerradas/page/10/',
 'https://www.fapesc.sc.gov.br/category/chamadas-encerradas/page/11/',
 'https://www.fapesc.sc.gov.br/category/chamadas-encerradas/page/12/',
 'https://www.fapesc.sc

In [21]:
fapesc= pd.DataFrame()

for link in links_pages_estados: 
    parser_pag_proyecto = parser(link)

    link_proyectos = parser_pag_proyecto.xpath('//div[@class="content"]//h2/a/@href')

    titulo_proyectos = parser_pag_proyecto.xpath('//div[@class="content"]//h2/a/text()')

    ano_bruto = parser_pag_proyecto.xpath('//p[@class="post-meta"]//span/text()')
    ano_proyectos = []
    for a in ano_bruto:
        a = a.split('de ')[2]
        ano_proyectos.append(a)

    if 'encerradas' in link:
        estado = 'Cerrada'
    if 'abertas' in link:
        estado = 'Abierta'
    if 'andamento' in link:
        estado = 'En evaluación'

    fapesc_pag = pd.DataFrame()
    fapesc_pag['Título'] = titulo_proyectos
    fapesc_pag['Año'] = ano_proyectos
    fapesc_pag['Link Proyecto'] = link_proyectos
    fapesc_pag['Estado'] = estado

    fapesc = pd.concat([fapesc, fapesc_pag])

fapesc= fapesc.reset_index(drop=True)

In [22]:
fapesc

,Título,Año,Link Proyecto,Estado
0,Suplemento de Chamada Pública Fapesc nº 18/202...,2021,https://www.fapesc.sc.gov.br/suplemento-de-cha...,Abierta
1,Suplemento de Chamada Pública Fapesc nº 17/202...,2021,https://www.fapesc.sc.gov.br/suplemento-de-cha...,Abierta
2,Edital de Chamada Pública Fapesc nº 16/2021 – ...,2021,https://www.fapesc.sc.gov.br/edital-de-chamada...,Abierta
3,Edital de Chamada Pública Fapesc nº 15/2021 – ...,2021,https://www.fapesc.sc.gov.br/edital-de-chamada...,Abierta
4,Edital de Chamada Pública Fapesc nº 14/2021 – ...,2021,https://www.fapesc.sc.gov.br/edital-de-chamada...,Abierta
...,...,...,...,...
144,Edital de Chamada Pública Fapesc nº 16/2020 – ...,2020,https://www.fapesc.sc.gov.br/edital-de-chamada...,En evaluación
145,Edital de Chamada Pública Fapesc nº 12/2020 – ...,2020,https://www.fapesc.sc.gov.br/edital-de-chamada...,En evaluación
146,Chamada Conjunta Internacional FAPESC/CONFAP/H...,2020,https://www.fapesc.sc.gov.br/chamada-conjunta-...,En evaluación
147,“Italy – Brazil Joint Science and Technology C...,2019,https://www.fapesc.sc.gov.br/confap-e-maeci-la...,En evaluación


In [32]:
descripcion_proyectos = []
pdfs_proyectos = []

for link in fapesc['Link Proyecto']:

    parser_proyecto = parser(link)

    descripcion_bruta = parser_proyecto.xpath('//div[@class="entry"]//p/text()')

    descripcion = ''
    for d in descripcion_bruta:
        descripcion = descripcion + d + ' '

    pdfs_bruta = parser_proyecto.xpath('//div[@class="entry"]//p/a/@href')

    pdfs = ''
    for p in pdfs_bruta:
        if p.endswith('.pdf'):
            pdfs = pdfs + p + ', '

    descripcion_proyectos.append(descripcion.strip(' '))
    pdfs_proyectos.append(pdfs.strip(', '))

fapesc['PDFs'] = pdfs_proyectos
fapesc['Descripción'] = descripcion_proyectos

In [38]:
fapesc.to_excel('brasil_fapesc.xlsx')